# Install and import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U "transformers>=4.41" peft datasets evaluate bitsandbytes sentencepiece rouge-score bert-score accelerate einops

In [ ]:
import random, torch, textwrap, evaluate, pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments, Seq2SeqTrainer,
    DataCollatorForSeq2Seq, BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, PeftModel
from tqdm.auto import tqdm

# PEFT

In [ ]:
SEED        = 42
MODEL_NAME  = "sshleifer/distilbart-cnn-12-6"
OUTPUT_DIR  = "/content/drive/MyDrive/nlp/distilbart_wikihow_headline_lora"
EPOCHS      = 1
BATCH       = 4
GRAD_ACC    = 4
MAX_SRC     = 165
MAX_TGT     = 12
LR          = 2e-4

random.seed(SEED); torch.manual_seed(SEED)

# Dataset split
df   = pd.read_csv("/content/drive/MyDrive/nlp/wikihow_clean.csv").drop(columns="Unnamed: 0")
data = Dataset.from_pandas(df)

tmp   = data.train_test_split(test_size=0.20, seed=SEED)
train = tmp["train"]
train = (train.shuffle(seed=SEED).select(range(30000)))
temp  = tmp["test"].train_test_split(test_size=0.50, seed=SEED)
val, test = temp["train"], temp["test"]

tok = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess(ex):
    src = ex["text"]
    tgt = ex["summary"]
    enc = tok(src, truncation=True, max_length=MAX_SRC)
    labels = tok(tgt, truncation=True, max_length=MAX_TGT)["input_ids"]
    enc["labels"] = labels
    return enc

train = train.map(preprocess, remove_columns=train.column_names, num_proc=4)
val   = val.map(preprocess,   remove_columns=val.column_names,   num_proc=4)

collator = DataCollatorForSeq2Seq(tok, model=None, padding="longest", return_tensors="pt")

bnb_cfg = BitsAndBytesConfig(load_in_8bit=True)

base = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_cfg,
    device_map={"": 0},
)
base.config.use_cache = False

lora_cfg = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.05,
    target_modules=["q_proj","v_proj","k_proj","out_proj"],
)
model = get_peft_model(base, lora_cfg)
model.print_trainable_parameters()

# Trainer
args = Seq2SeqTrainingArguments(
    OUTPUT_DIR,
    num_train_epochs       = EPOCHS,
    per_device_train_batch_size = BATCH,
    gradient_accumulation_steps = GRAD_ACC,
    learning_rate          = LR,
    fp16                   = True,
    dataloader_num_workers = 2,
    logging_steps          = 200,
    report_to              = "none",
    seed                   = SEED,

    eval_strategy          = "epoch",
    save_strategy          = "epoch",
    save_total_limit       = 1,
    logging_strategy       = "steps",
)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train,
    eval_dataset=val,
    data_collator=collator,
)
trainer.train()
trainer.save_model(OUTPUT_DIR)
tok.save_pretrained(OUTPUT_DIR)

print("✅ Adapter LoRA salvato in:", OUTPUT_DIR)

Map (num_proc=4):   0%|          | 0/30000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/12661 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


trainable params: 3,145,728 || all params: 308,656,128 || trainable%: 1.0192


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss
1,1.924600,No log


✅ Adapter LoRA salvato in: /content/drive/MyDrive/nlp/distilbart_wikihow_headline_lora


In [ ]:
SEED = 42
random.seed(SEED); torch.manual_seed(SEED)

MODEL_NAME  = "sshleifer/distilbart-cnn-12-6"
ADAPTER_DIR = "/content/drive/MyDrive/nlp/distilbart_wikihow_headline_lora"
MAX_SRC, MAX_TGT = 165, 12
BATCH_GEN = 32

tok   = AutoTokenizer.from_pretrained(ADAPTER_DIR)
base  = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME, device_map={"":0})
model = PeftModel.from_pretrained(base, ADAPTER_DIR)
model.eval()

# generation strategies
def gen(texts, mode):
    gen_kwargs = dict(max_length=MAX_TGT, do_sample=False)
    if mode == "greedy":
        gen_kwargs.update(num_beams=1)
    elif mode == "beam":
        gen_kwargs.update(num_beams=4)
    elif mode == "topk":
        gen_kwargs.update(do_sample=True, top_k=50, temperature=0.7, num_beams=1)
    elif mode == "topp":
        gen_kwargs.update(do_sample=True, top_p=0.9, temperature=0.7, num_beams=1)
    else:
        raise ValueError("unknown mode")

    inputs = tok(texts, truncation=True, max_length=MAX_SRC,
                 padding=True, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outs = model.generate(**inputs, **gen_kwargs)
    return tok.batch_decode(outs, skip_special_tokens=True)

decode_modes = ["greedy", "beam", "topk", "topp"]
preds_dict = {m: [] for m in decode_modes}
refs = []

for i in tqdm(range(0, len(test), BATCH_GEN)):
    batch = test[i: i+BATCH_GEN]
    refs.extend(batch["summary"])
    for m in decode_modes:
        preds_dict[m].extend(gen(batch["text"], m))

# metrics
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")
print("\n=== METRICHE ===")
for m in decode_modes:
    r = rouge.compute(predictions=preds_dict[m], references=refs)
    b = bertscore.compute(predictions=preds_dict[m], references=refs, lang="en")
    f1 = sum(b["f1"]) / len(b["f1"])
    print(f"\n→ {m.upper()}")
    for k,v in r.items():
        print(f"  {k:9s}: {v:.4f}")
    print(f"  BERTScore F1: {f1:.4f}")

# printing examples
print("\n=== ESEMPI ===")
sample_idx = random.sample(range(len(test)), 3)
for idx in sample_idx:
    art  = textwrap.shorten(test[idx]["text"], width=200, placeholder=" ...")
    gold = refs[idx]
    print(f"\n📄  Article: {art}")
    print(f"✅  Gold   : {gold}")
    for m in decode_modes:
        print(f"🤖  {m:<6}: {preds_dict[m][idx]}")


  0%|          | 0/396 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['early_stopping', 'length_penalty']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1642: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (12). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(
The following generation flags are not valid and may be ignored: ['early_stopping', 'length_penalty']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['early_stopping', 'length_penalty']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['early_stopping', 'length_penalty']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid a


=== METRICHE ===


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



→ GREEDY
  rouge1   : 0.4500
  rouge2   : 0.2485
  rougeL   : 0.4327
  rougeLsum: 0.4330
  BERTScore F1: 0.8997

→ BEAM
  rouge1   : 0.4607
  rouge2   : 0.2566
  rougeL   : 0.4432
  rougeLsum: 0.4434
  BERTScore F1: 0.9043

→ TOPK
  rouge1   : 0.4047
  rouge2   : 0.2048
  rougeL   : 0.3871
  rougeLsum: 0.3872
  BERTScore F1: 0.8933

→ TOPP
  rouge1   : 0.4212
  rouge2   : 0.2198
  rougeL   : 0.4030
  rougeLsum: 0.4031
  BERTScore F1: 0.8958

=== ESEMPI ===

📄  Article: Never let a used milk jug go to waste––in this article, you'll learn how to turn a used one into very handy planters for growing plants indoors.
✅  Gold   : Make a Planter out of an Old Milk Jug
🤖  greedy: Make a Milk Jug into a
🤖  beam  : Turn a Milk Jug into a Planter
🤖  topk  : Use a Used Milk Jar to Grow
🤖  topp  : Make a Milk Jug into a

📄  Article: Chibi is a Japanese word used to describe a "small thing or a person with a small body." Chibi is a simple, cute form of manga -- Super Deformed or SD is another name f

# Base model

In [ ]:
SEED = 42
random.seed(SEED); torch.manual_seed(SEED)

MODEL_NAME = "sshleifer/distilbart-cnn-12-6"
MAX_SRC, MAX_TGT = 165, 12
BATCH_GEN = 32

tok = AutoTokenizer.from_pretrained(MODEL_NAME)
bnb_cfg = BitsAndBytesConfig(load_in_8bit=True)

model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_cfg,
    device_map={"":0}
)
model.eval()

def gen(texts, mode):
    gen_kwargs = dict(max_length=MAX_TGT, do_sample=False)
    if mode == "greedy":
        gen_kwargs.update(num_beams=1)
    elif mode == "beam":
        gen_kwargs.update(num_beams=4)
    elif mode == "topk":
        gen_kwargs.update(do_sample=True, top_k=50, temperature=0.7, num_beams=1)
    elif mode == "topp":
        gen_kwargs.update(do_sample=True, top_p=0.9, temperature=0.7, num_beams=1)
    else:
        raise ValueError("unknown mode")

    inputs = tok(texts, truncation=True, max_length=MAX_SRC,
                 padding=True, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outs = model.generate(**inputs, **gen_kwargs)
    return tok.batch_decode(outs, skip_special_tokens=True)

decode_modes = ["greedy", "beam", "topk", "topp"]
preds_dict = {m: [] for m in decode_modes}
refs = []

for i in tqdm(range(0, len(test), BATCH_GEN)):
    batch = test[i : i+BATCH_GEN]
    refs.extend(batch["summary"])
    for m in decode_modes:
        preds_dict[m].extend(gen(batch["text"], m))

rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

print("\n=== METRICHE (modello base) ===")
for m in decode_modes:
    r = rouge.compute(predictions=preds_dict[m], references=refs)
    b = bertscore.compute(predictions=preds_dict[m], references=refs, lang="en")
    f1 = sum(b["f1"]) / len(b["f1"])
    print(f"\n→ {m.upper()}")
    for k,v in r.items():
        print(f"  {k:9s}: {v:.4f}")
    print(f"  BERTScore F1: {f1:.4f}")

print("\n=== ESEMPI ===")
sample_idx = random.sample(range(len(test)), 3)
for idx in sample_idx:
    art  = textwrap.shorten(test[idx]["text"], width=200, placeholder=" ...")
    gold = refs[idx]
    print(f"\n📄  Article: {art}")
    print(f"✅  Gold   : {gold}")
    for m in decode_modes:
        print(f"🤖  {m:<6}: {preds_dict[m][idx]}")


  0%|          | 0/396 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['early_stopping', 'length_penalty']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['early_stopping', 'length_penalty']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['early_stopping', 'length_penalty']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['early_stopping', 'length_penalty']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['early_stopping', 'length_penalty']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['early_stopping', 'length_penalty']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['ear


=== METRICHE (modello base) ===


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



→ GREEDY
  rouge1   : 0.2356
  rouge2   : 0.0928
  rougeL   : 0.2162
  rougeLsum: 0.2164
  BERTScore F1: 0.8514

→ BEAM
  rouge1   : 0.2300
  rouge2   : 0.0941
  rougeL   : 0.2143
  rougeLsum: 0.2145
  BERTScore F1: 0.8510

→ TOPK
  rouge1   : 0.2279
  rouge2   : 0.0869
  rougeL   : 0.2093
  rougeLsum: 0.2095
  BERTScore F1: 0.8507

→ TOPP
  rouge1   : 0.2347
  rouge2   : 0.0907
  rougeL   : 0.2153
  rougeLsum: 0.2155
  BERTScore F1: 0.8512

=== ESEMPI ===

📄  Article: Never let a used milk jug go to waste––in this article, you'll learn how to turn a used one into very handy planters for growing plants indoors.
✅  Gold   : Make a Planter out of an Old Milk Jug
🤖  greedy:  Never let a used milk jug go to waste
🤖  beam  :  Never let a used milk jug go to waste
🤖  topk  :  Use a used milk jug for a very handy
🤖  topp  :  Never let a used milk jug go to waste

📄  Article: Chibi is a Japanese word used to describe a "small thing or a person with a small body." Chibi is a simple, cute form 